# Análise das notícias sob a ótica ESG: geração da base com sentimentos

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG

In [1]:
import warnings
import re
import pandas as pd
import numpy as np
import datetime  as dt
from noticias_processamento_texto import *
from analise_sentimento_modelo_gpt import *
from classificador_esg import aplica_classificador_esg
import pickle
from cotacoes import *
from noticias_io import *

warnings.filterwarnings('ignore')

/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Us

## Lendo base de notícias

In [2]:
df = le_base_noticias_bruta_para_df()

df['fonte'] = df['fonte'].apply(trata_nome_fontes)
df['data_publicacao'] = df['data_publicacao'].dt.date
df = df.sort_values(by=['empresa', 'data_publicacao'])
df['titulo_par'] = df['titulo'].str.strip().apply(remove_acentos).str[:15]
df = df.drop_duplicates(subset=['empresa', 'fonte', 'data_publicacao', 'titulo_par'], keep='first')
del df['titulo_par']

# ajustes
df = ajusta_nomes_empresas_dataframe(df)

# cria hash
df['hash'] = df.apply(lambda row: criar_hash_noticia(row['texto_completo'], row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)

# le lista empresas
df_empresas = le_lista_empresas_para_df()

## Aplicação Classificador SVM

In [3]:
# utiliza o classificador por ML para determinar o assunto da noticia: E, S, G ou outros
vect = pickle.load(open('models/svm_vetorizador.sav', 'rb'))
model = pickle.load(open('models/svm_classificador.sav', 'rb'))

if len(df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo']))), 'classificacao_ml']) >0:
    df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo']))), 'classificacao_ml'] = aplica_classificador_esg(vect, model, 
                                                                                                              df.loc[ (pd.isnull(df['classificacao_ml']) &   (~pd.isnull(df['texto_completo'])))], 
                                                                                                              comparar_com_real=False, col_texto_origem='texto_completo', 
                                                                                                              col_texto_saida='texto_ajustado', col_classe_verdadeira='classificacao')       
    
# salva na base original para cache
salva_base_noticias_bruta(df)

['E' 'G' 'Outros' 'S']
[0.25568405 0.10168746 0.42157758 0.22105091]
[1 3 0 2]
0
['E' 'G' 'Outros' 'S']
[0.10179023 0.42312524 0.45906843 0.0160161 ]
[3 0 1 2]
1
['E' 'G' 'Outros' 'S']
[0.27576004 0.3082335  0.3498256  0.06618086]
[3 0 1 2]
1
['E' 'G' 'Outros' 'S']
[0.38943484 0.0933052  0.4946246  0.02263536]
[3 1 0 2]
0
['E' 'G' 'Outros' 'S']
[0.04449474 0.35379118 0.49750696 0.10420711]
[0 3 1 2]
1
['E' 'G' 'Outros' 'S']
[0.21027229 0.11005053 0.42311063 0.25656655]
[1 0 3 2]
3
['E' 'G' 'Outros' 'S']
[0.19922374 0.09217691 0.40054184 0.30805751]
[1 0 3 2]
3
['E' 'G' 'Outros' 'S']
[0.14134733 0.34618481 0.4923382  0.02012966]
[3 0 1 2]
1
['E' 'G' 'Outros' 'S']
[0.02024243 0.15432352 0.49330245 0.3321316 ]
[0 1 3 2]
3
['E' 'G' 'Outros' 'S']
[0.30350382 0.09872146 0.3209707  0.27680402]
[1 3 0 2]
0
['E' 'G' 'Outros' 'S']
[0.07448054 0.12188732 0.42537065 0.37826149]
[0 1 3 2]
3
['E' 'G' 'Outros' 'S']
[0.14165166 0.29791607 0.43003751 0.13039476]
[3 0 1 2]
1
['E' 'G' 'Outros' 'S']
[0.37

## Aplicação do CHAT GPT

In [4]:
df = le_base_noticias_bruta_para_df()


# abre o cache do chat gpt
df_cache = le_cache_gpt_para_df()
dic_cache = df_cache.set_index('hash')['resposta'].to_dict()
tam_inicial = len(dic_cache)


try:
    df['gpt_resposta_completa'] = df.apply(lambda row: classifica_sentimento_noticia_gpt(row['data_publicacao'], row['titulo'], row['texto_completo'], row['empresa'], dicionario_cache=dic_cache)  
                                                 if (  (not pd.isnull(row['texto_completo'])) and row['classificacao_ml'] != 'Outros' 
                                                         and ( pd.isnull(row['gpt_resposta_completa']))  
                                                         and (  pd.isnull(row['noticia_selecionada']) or row['noticia_selecionada']==1   ) ) 
                                                         else row['gpt_resposta_completa'], axis=1)
finally:                                                                    
    #salva o cache do gpt
    tam_final = len(dic_cache)
    print('Realizadas ' + str(tam_final - tam_inicial) + ' chamadas à API do GPT.')
    df_cache = pd.DataFrame({'hash': dic_cache.keys(),  'resposta': dic_cache.values()})
    salva_cache_gpt(df_cache)

# salva processamento na  base original
salva_base_noticias_bruta(df)


# faz os filtros
df = df[  (  (pd.isnull(df['noticia_selecionada']))  |  (df['noticia_selecionada'] ==1)  )  ]
df = df[~pd.isnull(df['texto_completo'])]
df = df[df['classificacao_ml'] != 'Outros']  

# tratando
df = gera_colunas_gpt(df, coluna_resposta_gpt='gpt_resposta_completa')
df = filtros_pos_gpt(df)
df['polaridade'] = df['gpt_polaridade']  # substitui metodo anterior
# resumos vazios
df['gpt_resumo'] = df.apply(lambda row: row['titulo'] if pd.isnull(row['gpt_resumo']) else row['gpt_resumo'] , axis=1)

df['gpt_lista_respostas'] = df['gpt_lista_respostas'].astype(str)  # para permitir remover duplicatas

# filtro lexico aplicados apenas as  novas noticias
df_filtrado = filtrar_noticias_pos_coleta_modelo_simplificado(df[pd.isnull(df['noticia_selecionada'])], df_empresas)

# salva o resultado do filtro por eficiencia (salva na base sem sentimentos por eficiencia)
# esse trecho permite que a rotina so seja aplicada a novas noticias futuramente, deixando proc. mais eficiente
df_original = le_base_noticias_bruta_para_df()
df_original['noticia_selecionada'] =  ( (df_original['noticia_selecionada']==1) |  df_original['hash'].isin(  df_filtrado['hash'].values  ) ).apply(lambda x: 1 if x else 0)
salva_base_noticias_bruta(df_original)
###


# aplica o filtro na base ja com sentimentos
df['noticia_selecionada'] =  ( (df['noticia_selecionada']==1) |  df['hash'].isin(  df_filtrado['hash'].values  ) ).apply(lambda x: 1 if x else 0)
df = df[df['noticia_selecionada']==1] 

# retira duplicatas
df = df.drop_duplicates(['hash'], keep='first')
df = df.sort_values(by=['Nome', 'data_publicacao'])

# salva a base de saida
salva_base_noticias_processada(df)

# versao reduzida para o site
df_short = df.loc[: , ['hash', 'titulo', 'data_publicacao', 'url', 'fonte', 'empresa', 'Nome',  'CNPJ', 'Razão social', 'Setor', 'Código',  'classificacao', 'gpt_resumo', 'polaridade']]
salva_base_noticias_compacta(df)

print('Processamento concluído')

Realizadas 388 chamadas à API do GPT.
Processamento concluído
